# PharmSci 175/275 (UCI)
## What is this?? 
The material below is Lecture 2 (on Energy Minimization) from Drug Discovery Computing Techniques, PharmSci 175/275 at UC Irvine. 
Extensive materials for this course, as well as extensive background and related materials, are available on the course GitHub repository: [github.com/mobleylab/drug-computing](https://github.com/mobleylab/drug-computing)

This material is a set of slides intended for presentation with RISE as detailed [in the course materials on GitHub](https://github.com/MobleyLab/drug-computing/tree/master/uci-pharmsci/lectures/energy_minimization). While it may be useful without RISE, it will also likely appear somewhat less verbose than it would if it were intended for use in written form.

# Energy landscapes and energy minimization

Today: Energy landscapes, energy functions and energy mimimization

### Instructor: David L. Mobley

### Contributors to today's materials:
- David L. Mobley
- [Previous contributions](https://engineering.ucsb.edu/~shell/che210d/) from M. Scott Shell (UCSB); some images used here are drawn from his materials.

## Energy landscapes provide a useful conceptual device

### Energy landscapes govern conformations and flexibility

### Chemistry takes place on energy landscapes
<div style="float: right">
    <img src="images/funnel.png" alt="GitHub" style="width: 400px;" align="right"/>
</div>

- Reactions and barriers
- Conformational change
- Binding/association


### Energy landscapes govern dynamics and thermodynamics
<div style="float: right">
    <img src="images/funnel.png" alt="GitHub" style="width: 400px;" align="right"/>
</div>
- Reaction rates
- Equilibrium properties

## Often, we are interested in exploring the energy landscape

A potential, $U(\bf{r^N})$ describes the energy as a function of the coordinates of the particles; here ${\bf r}$ is the coordinates, **boldface** denotes it is a vector, and the superscript denotes it is coordinates of all $N$ particles in the system.
<div style="float: right">
    <img src="images/funnel_labeled.png" alt="GitHub" style="width: 400px;" align="right"/>
</div>

### Landscapes have many features, including
- Global minimum, the most stable state (caveat: entropy)
- Local minima: other stable/metastable states

<div style="float: right">
    <img src="images/funnel_labeled.png" alt="GitHub" style="width: 1000px;" align="right"/>
</div>

### We can't visualize 3N dimensions, so we often project onto fewer dimensions

<div style="float: right">
    <img src="images/landscape_1D.png" alt="GitHub" style="width: 1000px;" align="right"/>
</div>

### Background: Vector notation

- For a single particle, we have coordinates $x$, $y$, and $z$, or $x_1$, $y_1$, and $z_1$ if it is particle 1
- We might write these as $(-1, 3, 2)$ for example, if $x=-1$, $y=3$, $z=2$. 
- For evem two particles, we have $x_1$ and $x_2$, $y_1$ and $y_2$, etc. 
- Writing out names of coordinates becomes slow, e.g. $f(x_1, y_1, z_1, x_2, y_2, z_2, ... z_N)$
- We simplify by writing $f({\bf r}^N)$ and remember that ${\bf r}^N$ really means:
\begin{equation}
{\bf r}^N=
\begin{bmatrix}
x_1 & y_1 & z_1 \\
x_2 & y_2 & z_2 \\
... & ... & ... \\
x_N & y_N & z_N \\
\end{bmatrix}
\end{equation}

## Forces are properties of energy landscapes, too

The force is the slope (technically, gradient):

$f_{x,i} = -\frac{\partial U({\bf r^N})}{\partial x_i}$, $f_{y,i} = -\frac{\partial U({\bf r^N})}{\partial y_i}$, $f_{z,i} = -\frac{\partial U({\bf r^N})}{\partial z_i}$

As shorthand, this may be written ${\bf f}^N = -\frac{\partial U({\bf r^N})}{\partial {\bf r^N}}$ or ${\bf f}^N = -\nabla \cdot U({\bf r^N})$ where the result, ${\bf f}^N$, is an Nx3 array (matrix, if you prefer)

If energy function is pairwise additive, can evaluate via summing individual interactions -- force on atom k is 
\begin{equation}
{\bf f}_k = \sum_{j\neq k} \frac{ {\bf r}_{kj}}{r_{kj}}  \frac{\partial}{\partial r_{kj}} U(r_{kj})
\end{equation} where ${\bf r_{kj}} = {\bf r}_j - {\bf r_k}$. Note not all force calculations are necessary: ${\bf f}_{kj} = -{\bf f}_{jk}$

## The matrix of second derivatives is called the Hessian and distinguishes minima from saddle points



\begin{equation}
{\bf H}({\bf r}^N)=
\begin{bmatrix}
\frac{d^2 U({\bf r^N}) }{ d x_1^2} & \frac{d^2 U({\bf r^N}) }{ dx_1 d y_1} & ... & \frac{d^2 U({\bf r^N}) }{ dx_1 d z_1} \\
\frac{d^2 U({\bf r^N}) }{ d y_1 d x_1} & \frac{d^2 U({\bf r^N}) }{ d y_1^2} & ... & \frac{d^2 U({\bf r^N}) }{ dy_1 d z_N}\\
... & ... & ... & ... \\
\frac{d^2 U({\bf r^N}) }{ d z_N d x_1} & \frac{d^2 U({\bf r^N}) }{ d z_N d y_1} & ... & \frac{d^2 U({\bf r^N}) }{ dz_N^2}\\
\end{bmatrix}
\end{equation}

## Types of stationary points can be distiguished from derivatives

- Stationary points have zero force on each particle: $ \nabla \cdot U({\bf r^N}) = {\bf 0}$
- These can be minima or maxima
- Minima have negative curvature in all directions (restoring force is towards the minimum)


## Energy landscapes have *lots* of minima

(Update with example disconnectivity graph here if possible)

e.g. Doye, Miller, and Wales, J. Chem. Phys. 111: 8417 (1999)

- Disconnectivity graph shows minima for 13 LJ atoms
- To move between two minima, have to go to point where lines from two minima reach same energy
- 1467 distinct minima for 13 atoms!

## We care a lot about finding minima

- As noted, gives a first guess about most stable states
- Minima are stable structures, point of contact with experiment
- Initial structures need to be relaxed so forces are not too large
  - Remove strained bonds, atom overlaps
- Minimization is really optimization:
  - If you can find the minimum of $U$, you can find the minimum of $-U$
  - Same techniques apply to other things, i.e. finding set of parameters that minimizes an error, etc.

### Findining minima often becomes a numerical task, because analytical solutions become impractical very quickly

Consider $U(x,y) = x^2 + (y-1)^2$ for a single particle in a two dimensional potential. Finding $\nabla\cdot U = 0$ yields:

$2x = 0$ and $2(y-1)=0$ or $x=0$, $y=1$

simple enough

But in general, N dimensions means N coupled, potentially nonlinear equations. Consider 
\begin{equation}
U = x^2 z^2 +x (y-1)^2 + xyz + 14y + z^3
\end{equation}
Setting the derivatives to zero yields: 

$0 = 2xz^2 + (y-1)^2 +yz$

$0 = 2x(y-1) + xz + 14$

$0 = 2x^2z + xy + 3z^2$

**Volunteers??** It can be solved, but not fun.

**And this is just for a single particle in a 3D potential**, so we are typically forced to numerical minimizations, even when potential is analytic

### Energy minimization is a sub-class of the more general problem of finding roots

Common problem: For some $f(x)$, find values of $x$ for which $f(x)=0$

Many equations can be re-cast this way. *i.e.*, if you need to solve $g(x) = 3$, define $f(x) = g(x)-3$ and find $x$ such that $f(x)=0$

If $f(x)$ happens to be the force, this maps to energy minimization

As a consequence: Algorithms used for energy minimization typically have broader application to finding roots

## Steepest descents is a simple minimization algorithm that always steps as far as possible along the direction of the force
Take ${\bf f}^N = -\frac{\partial U({\bf r}^N)}{\partial {\bf r}^N}$, then:
1. Move in direction of last force until the minimum *in that direction* is found
2. Compute new ${\bf f}_i^N $ for iteration $i$, perpendicular to previous force

<div style="float: right">
    <img src="images/steepest_descent.svg" alt="GitHub" style="width: 300px;" align="right"/>
</div>
Repeat until minimum is found

**Limitations**: 

Oscillates in narrow valleys; slow near minimum.

<div style="float: right">
    <img src="images/steepest_descent.svg" alt="GitHub" style="width: 700px;" align="right"/>
</div>

### Steepest descents oscillates in narrow valleys and is slow near the minimum
<div style="float: center">
    <img src="images/Banana-SteepDesc.gif" style="width: 600px;" align="center"/>
</div>

(Illustration, P.A. Simonescu, [Wikipedia](https://en.wikipedia.org/wiki/Gradient_descent#/media/File:Banana-SteepDesc.gif), [CC-BY-SA](https://creativecommons.org/licenses/by-sa/3.0/))

### Another illustration further highlights this

(In this case, steepest *ascents*, but it's just the negative...)
<div style="float: left">
    <img src="images/gradient_ascent_contour.png" style="width: 400px;" align="left"/>
</div>
<div style="float: right">
    <img src="images/gradient_ascent_surface.png" style="width: 500px;" align="right"/>
</div>


(Images public domain: [source](https://upload.wikimedia.org/wikipedia/commons/d/db/Gradient_ascent_%28contour%29.png), [source](https://upload.wikimedia.org/wikipedia/commons/6/68/Gradient_ascent_%28surface%29.png))

## A line search can make many minimization methods more efficient

A line search is an efficient way to find a minimum along a particular direction
- Line search: Bracket minimum
  - Start with initial set of coordinates ${\bf r}$ and search direction ${\bf v}$ that is downhill
  - Generate pairs of points a distance $d$ and $2d$ along the line (${\bf r} + d{\bf v}, {\bf r}+2d{\bf v}$)
     1. If the energy at the further point is higher than the energy at the nearer point, stop
     2. Otherwise, move the pair of points $d$ further along the line and go back to 1.

### To finish a line search, identify the minimum precisely

- Fit a quadratic to our 3 points (initial, and two bracket points)
- Guess that minimum is at the zero of the quadratic, call this point 4.
- Fit a new quadratic using points 2, 3, and 4, and move to the zero.
- Repeat until the energy stops changing within a given tolerance

## To do better than steepest descents, let's consider its pros and cons

<div style="float: right">
    <img src="images/gradient_ascent_contour.png" style="width: 550px;" align="right"/>
</div>

- Good to go in steepest direction initially 
- It’s a good idea to move downhill
- It is initially very fast (see Leach Table 5.1)
- But steepest descent overcorrects 


### Conjugate gradient (CG) is more efficient; it works like steepest descent but chooses a different direction

- Start with an initial direction ${\bf v}$ that is downhill, move in that direction until a minimum is reached.

- Compute a new direction using ${\bf v}_i = {\bf f}^N_i + \gamma_i {\bf v}_{i-1}^N$ where $\gamma_i = \frac{({\bf f}^N_i-{\bf f}^N_{i-1}){\bf f}^N_i}{{\bf f}^N_i {\bf f}^N_i}$; note $\gamma_i$ is a scalar.

Note that by ${\bf f}^N {\bf f}^N$ we mean vector multiplication, not a matrix multiplication; that is:
\begin{equation}
{\bf f}^N {\bf f}^N = f^2_{x,1} + f^2_{y,1} + f^2_{z,1} + f^2_{x,2} + ... + f^2_{z, N}
\end{equation}

(In Python, this can be coded by multiplying ${\bf f}\times {\bf f}$, where ${\bf f}$ are arrays, and taking the sum of the result; for normal matrix multiplication of arrays one would use a dot product)

$\gamma_i$ is designed so that the new direction is *conjugate* to the old direction so that the new step does not undo any of the work done by the old step (causing oscillation)

## Conjugate gradient (CG) is more efficient; it works like steepest descent but chooses a different direction

<div style="float: right">
    <img src="images/conjugate_gradient_illustration.svg" style="width: 300px;" align="right"/>
</div>
- Green: Steepest descent (with optimal step size)
- Red: Conjugate gradient

Takes at most $M$ steps, where $M$ is the number of degrees of freedom, often $3N$


(Image: [Wikipedia](https://en.wikipedia.org/wiki/Conjugate_gradient_method#/media/File:Conjugate_gradient_illustration.svg), public domain. Oleg Alexandrov.) 

## More advanced minimization methods can have considerable advantages

- Newton-Raphson: Based on Taylor expansion of potential around zero
  - reaches minimum in one step for quadratic potentials
  - converges to minima and saddle points, so requires initial moves to reach minima
  - slow because requires Hessian at every step
- Quasi-Newton methods approximate the Hessian to go faster
  - L-BFGS is a popular quasi-Newton method

## Locating global minima is challenging and often involves combination of techniques

- Need some way to cross barriers, in addition to minimization
- Often combine techniques
  - i.e. Molecular Dynamics or Monte Carlo + minimization
- Normal mode analysis can be used to identify collective motions
  - Once identified, system can be moved in those directions